In [1]:
import re
import nltk
import requests
import warnings
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

### Aux Functions 

In [2]:
def get_rows_list(full_rows_list) -> tuple:
    full_pre_list = []
    full_discarted_list = []
    
    for i in range(0, len(full_rows_list)):
        row = full_rows_list[i]

        if row:
            if row[0].startswith('(') and len(row) == 1:
                full_discarted_list.append(row)

            if len(row[0]) == 1 and len(row) == 1 and row[0].startswith("("):
                full_discarted_list.append(row)

            elif row[-1].upper() == "(VIGÊNCIA)":
                full_pre_list.append(row[:-1])

            elif row[0].startswith('(revogado') or 'vigência encerrada' in ' '.join(row[:-1]):
                full_discarted_list.append(row)

            elif row[0].endswith('(revogado).') or row[0].endswith('(revogado);'):
                full_discarted_list.append(row)

            elif row[0].startswith('vigência encerrada') or row[0].startswith('(vigência encerrada)'):
                full_discarted_list.append(row)

            else:
                full_pre_list.append(row)
                
    return (full_pre_list, full_discarted_list)

# nltk.download('punkt')
# nltk.download('maxent_ne_chunker')
# warnings.filterwarnings('ignore')

# 1.0. CLT Data Collect

## Data Collect

In [3]:
r_list = ['I', 'IL', 'II','III','IV','V','VI','VII','VIII','IX','X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII','XIX','XX','XXI','XXII','XXIII','XXIV','XXV','XXVI','XXVII','XXVIII','XXIX','XXX','XXXI','XXXII','XXXIII','XXXIV','XXXV','XXXVI','XXXVII','XXXVIII','XXXIX','XL','XLI','XLII','XLIII','XLIV','XLV','XLVI','XLVII','XLVIII','XLIX','L','LI',
          'LII','LIII','LIV','LV','LVI','LVII','LVIII','LIX','LX','LXI','LXII','LXIII','LXIV','LXV','LXVI','LXVII','LXVIII','LXIX','LXX','LXXI','LXXII','LXXIII','LXXIV','LXXV','LXXVI','LXXVII','LXXVIII','LXXIX','LXXX','LXXXI','LXXXII','LXXXIII','LXXXIV','LXXXV','LXXXVI','LXXXVII','LXXXVIII','LXXXIX','XC','XCI','XCII','XCIII','XCIV','XCV','XCVI','XCVII','XCVIII','XCIX','C']

alineas_list = [
    'a)', 'b)', 'c)', 'd)', 'e)', 'f)', 'g)', 'h)', 'i)', 'j)', 'k)', 'l)', 'm)',
    'n)', 'o)', 'p)', 'q)', 'r)', 's)', 't)', 'u)', 'v)', 'w)', 'x)', 'y)', 'z)'
]

In [4]:
soup = BeautifulSoup(open('page/clt.html').read(), 'html.parser')

In [5]:
source = ' '.join([k.strip() for k in soup.find('big').get_text().split('\n')])

In [6]:
#full_html_page_list = soup.find_all('p')[8:]

full_html_page = soup.find('body')

strike_tag = full_html_page.find_all('strike')
st = [k.extract() for k in full_html_page.find_all('strike')]

In [7]:
full_html_page_list = full_html_page.find_all('p')

for k in range(0, 20):
    if full_html_page_list[k].get_text().startswith('O PRESIDENTE'):
        full_html_page_list = full_html_page_list[k:]
        break

In [8]:
full_rows_list = []

for page in full_html_page_list:
    txt = page.get_text().replace('rt.', 'rt').replace('arts.', 'arts ').replace('\xa0', '\n').replace('\n\t', '').replace('\t', '')
    txt = txt.replace('º', '.').lower().replace('(revogada', '(revogado')
    
    for j in range(1, 922): # Manual Index :(
        txt = txt.replace(f'{j}º', f'{j}.').replace(f'{j}o', f'{j}.').replace(f'{j}ª ', f'{j}. ').replace(f'{j}) ', f'{j}. ')
    
    pre_txt = [k.strip() for k in nltk.tokenize.line_tokenize(txt)]
    full_rows_list.append(pre_txt)
    
full_pre_list, full_discarted_list = get_rows_list(full_rows_list)

In [9]:
arts = []
discarted_arts = []
for i in range(0, len(full_pre_list)):
    row = full_pre_list[i]
    
    if row[0].startswith('art'):
        string_row = ' '.join(row).replace(' - ', '. ').replace('art .', 'art').replace('.. ', '. ')
        string_token = nltk.tokenize.wordpunct_tokenize(string_row)
        texto = ' '.join(string_token[2:])
        
        try:
            if string_token[2] == '-' and len(string_token[3]) == 1:
                artigo = string_token[0]+' '+''.join(string_token[1:4]).upper()
            elif len(string_token[0]) > 3:
                artigo = ' '.join(string_token[:2]).replace('art', 'art ').strip()+'.'
            else: 
                artigo = ' '.join(string_token[:2]).strip()+'.'
        except:
            artigo = ' '.join(string_token)
        
        arts.append(
            {'artigo': artigo, 'texto': texto}
        )
        
    elif row[0].startswith('§'):
        string_row = ' '.join(row).replace(' -', '').replace('.-', '-')
        string_token = nltk.tokenize.wordpunct_tokenize(string_row)
        
        if string_token[2] == '-':
            para = string_token[0]+' '+''.join(string_token[1:4]).upper()
            texto = ' '.join(string_token[5:]).strip().capitalize()
            
        else:
            para = ' '.join(string_token[:2])+'.'
            texto = ' '.join(string_token[2:]).strip().capitalize()
            
        arts.append(
            {'paragrafo': para, 'texto': texto}
        )
        
    elif row[0].split()[0] in alineas_list:
        string_token = nltk.tokenize.wordpunct_tokenize(' '.join(row)) 
        arts.append(
            {'alinea': ''.join(string_token[0:2]), 'texto': ' '.join(string_token[2:]).capitalize() }
        )
    
    elif row[0].upper().split()[0] in r_list:
        string_row = ' '.join(row).replace(' - ', '. ').split()
        
        arts.append(
            {'inciso': string_row[0], 'texto': ' '.join(string_row[1:])}
        )
    
    elif row[0].startswith('parágrafo único') or ' '.join(row[:1]) == 'parágrafo único':
        string_row = ' '.join(row).replace(' - ', '.')
        texto = ' '.join(string_row.split()[2:]).strip().capitalize()
        
        arts.append(
            {'paragrafo_unico': 'pu', 'texto': texto}
        )
        
    elif row[0].startswith('título'): #or row[0].startswith('seção'):
        info = ' '.join([k.strip() for k in row[0].split()]).upper()
        title_content = ' '.join(full_pre_list[i+1])
        if title_content.startswith('art'):
            title_content = 'NAN'
        
        content = info + ' ' + title_content.upper()
        
        arts.append({'titulo': content})
        
    else:
        discarted_arts.append(row)

In [10]:
df_raw = pd.DataFrame(arts)

In [11]:
df_raw.head()

,artigo,texto,paragrafo_unico,titulo,paragrafo,inciso,alinea
0,art 1.,. fica aprovada a consolidação das leis do tra...,NaN,NaN,NaN,NaN,NaN
1,NaN,Continuam em vigor as disposições legais trans...,pu,NaN,NaN,NaN,NaN
2,art 2.,. o presente decreto - lei entrará em vigor em...,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,TÍTULO I INTRODUÇÃO,NaN,NaN,NaN
4,art 1.,. esta consolidação estatui as normas que regu...,NaN,NaN,NaN,NaN,NaN


## Data Cleaning

In [12]:
df = df_raw.copy()

df1 = df.drop_duplicates().reset_index(drop=True)

### Arts Cleaning

In [13]:
df1.artigo = df1.artigo.fillna(method='ffill')
df1.artigo = [k[:3].capitalize()+'.'+k[3:] for k in df1.artigo.tolist()]

### Unique Parag, Inciso and Alinea Cleaning

In [14]:
df1.paragrafo = df1.paragrafo.fillna('na')
df1.inciso = df1.inciso.fillna('na')
df1.alinea = df1.alinea.fillna('na')
df1.paragrafo_unico = df1.paragrafo_unico.apply(lambda x: 'PU' if x == 'pu' else 'na')

In [15]:
df1.head(3)

,artigo,texto,paragrafo_unico,titulo,paragrafo,inciso,alinea
0,Art. 1.,. fica aprovada a consolidação das leis do tra...,na,NaN,na,na,na
1,Art. 1.,Continuam em vigor as disposições legais trans...,PU,NaN,na,na,na
2,Art. 2.,. o presente decreto - lei entrará em vigor em...,na,NaN,na,na,na


### Title Cleaning

In [16]:
df1.titulo = df1.titulo.fillna(method='ffill')

### Text Cleaning

In [17]:
df1 = df1[~df1.texto.isna()].reset_index(drop=True)

In [18]:
df1.texto = [k[2:].strip() if k.startswith('.') else k for k in df1.texto.tolist()]
df1.texto = [k[5:].strip() if k.startswith('- ') else k for k in df1.texto.tolist()]
df1.texto = df1.texto.apply(lambda x: x.lower().strip().replace('revogada', 'revogado'))

df1 = df1[~df1.texto.str.contains('revogado')].reset_index(drop=True)
df1.texto = df1.texto.apply(lambda x: x.strip().capitalize().replace(' - ', '-').replace(' ,', ',').replace(' ;', ';').replace(' .', '.').replace(' :', ':'))
df1.texto = df1.texto.apply(lambda x: x+'.' if not (x.endswith('.') or x.endswith(';')) else x)

## Data Verify

In [19]:
df2 = df1.copy()

df2 = df2.iloc[:-1, :]

### Ordering Columns

In [20]:
df2['sigla'] = 'CLT'
df2['link']  = 'http://www.planalto.gov.br/ccivil_03/decreto-lei/del5452.htm'
df2['scrapy_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
df2['process_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [21]:
df2.columns.tolist()

['artigo',
 'texto',
 'paragrafo_unico',
 'titulo',
 'paragrafo',
 'inciso',
 'alinea',
 'sigla',
 'link',
 'scrapy_datetime',
 'process_datetime']

In [23]:
df2 = df2[['sigla', 'titulo', 'texto', 'artigo', 'inciso', 'alinea', 'paragrafo', 'paragrafo_unico', 
           'link', 'scrapy_datetime', 'process_datetime']]

### Save to CSV

In [24]:
df2.to_csv(f"clt_scrapy_{datetime.now().strftime('%Y-%m-%d')}.csv", index=False)

## Streamlit App Data Transform

In [25]:
import re
import pandas as pd 

df = pd.read_csv('clt_scrapy_2022-09-08.csv')
df = df.drop_duplicates().reset_index(drop=True)

Pattern: CF88 - Art. 1. § 9 a) III PU

In [26]:
df['padrao'] = df.apply(lambda x: (x['sigla'] + ' - ' + x['artigo'] + ' ' + 
                                   x['paragrafo'].replace('na', '') + ' ' + 
                                   x['alinea'].replace('na', '') + ' ' + 
                                   x['inciso'].replace('na', '') + ' ' + 
                                   x['paragrafo_unico'].replace('na', '').upper()).strip(), axis=1)

df['padrao'] = df['padrao'].apply(lambda x: ' '.join(x.split()))

In [28]:
df[['titulo', 'padrao', 'texto', 'link']].to_csv('clt_elastic.csv', index=False)